<a href="https://colab.research.google.com/github/Pavan-pk/Kaggle/blob/main/Kaggle_cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Log in to kaggel, get the API token and upload.
from google.colab import files
files.upload()

In [ ]:
!ls -lha kaggle*

In [ ]:
!pip install -q Kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s Dogs

In [ ]:
!kaggle datasets download -d thesherpafromalabama/cats-and-dogs-sentdex-tutorial

In [ ]:
!ls -lha 

In [ ]:
!unzip cats-and-dogs-sentdex-tutorial.zip -d ./sentdex-tutorial > unwanted_unzip_log

In [ ]:
!ls -l

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
DATADIR = "./sentdex-tutorial/PetImages"
CATEGORIES = ["Dog", "Cat"]

In [ ]:
training_data = []
def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (128, 128))
        training_data.append([new_array, class_num])
      except:
        pass

create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random
random.shuffle(training_data)

In [ ]:
X = []
Y = []

for features, labels in training_data:
  X.append(features)
  Y.append(labels)

X = np.array(X).reshape(-1, 128, 128, 1)
X = X/255.0

In [ ]:

!apt install cuda

In [ ]:
!pip install -q tf-nightly-gpu-2.0-preview
%load_ext tensorboard

In [ ]:
!pip install tensorboardcolab

In [ ]:
import tensorflow as tf
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
!rm -rf logs_multiple

In [ ]:
%reload_ext tensorboard


In [ ]:

dense_layers = [1, 2, 3]
layer_sizes = [64, 128, 256]
conv_layers = [2, 3, 4]
for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
      print("-------------------------------------------------------------------")
      print(NAME)
      print("-------------------------------------------------------------------")
      model = Sequential()
      
      model.add(Conv2D(layer_size, (3,3),input_shape=X.shape[1:])) #input_shape is image resolution
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2,2)))
      for i in range(conv_layer):
        print("added conv layer")
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))

      model.add(Flatten())
      for j in range(dense_layer):
        print("added dense layer")
        model.add(Dense(layer_size))
        model.add(Activation("relu"))

      model.add(Dense(1))
      model.add(Activation("sigmoid"))
      tensorboard = TensorBoard(log_dir='logs_multiple/{}'.format(NAME))
      model.compile(loss="binary_crossentropy",
                   optimizer="adamax",
                   metrics=["accuracy"])

      model.fit(X, Y, batch_size=32, validation_split=0.2, epochs=15, callbacks=[tensorboard])

#Read tensor.keras.callbacks

In [ ]:
  %tensorboard --logdir logs_multiple
 